# Building the model

The purpose of this notebook is to build the model that will eventually be used as the model behind the PL prediction for the app

## Imports

In [ ]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf

## Loading the dataset

In [ ]:
combined = pd.read_csv("./final_combined_dataframe.csv")

In [ ]:
combined.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1927 entries, 0 to 1926
Data columns (total 69 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   goals                               1927 non-null   float64
 1   assists                             1927 non-null   float64
 2   non_penalty_goals                   1927 non-null   float64
 3   penalties_scored                    1927 non-null   float64
 4   penalties_attempted                 1927 non-null   float64
 5   yellow_cards                        1927 non-null   float64
 6   red_cards                           1927 non-null   float64
 7   expected_goals                      1927 non-null   float64
 8   non_penalty_expected_goals          1927 non-null   float64
 9   expected_assisted_goals             1927 non-null   float64
 10  progressive_carries                 1927 non-null   float64
 11  progressive_passes                  1927 no

In [ ]:
combined.head()

## Scaling

In [ ]:
scaler = StandardScaler(copy=True)

In [ ]:
combined_standardized = scaler.fit_transform(combined[pure_stats_columns_no_minutes])

In [ ]:
evr = {}
n_components = {}
feature_to_pc = {}

for n in range(1,26):
	pca = PCA(n_components = n, random_state=938)
	pca.fit(combined_standardized)
	feature_to_pc_map = pd.DataFrame(pca.components_, columns=pure_stats_columns_no_minutes)
	components = pca.transform(combined_standardized)
	components_df = pd.DataFrame(data=components[:, [p for p in range(n)]], columns=pca.get_feature_names_out(), )
	
	evr[n] = sum(pca.explained_variance_ratio_)
	n_components[n] = components_df
	feature_to_pc[n] = feature_to_pc_map

pd.DataFrame(data=evr, index=["explained_variance_ratio"]).T

,explained_variance_ratio
1,0.686249
2,0.801371
3,0.860920
4,0.885021
5,0.902991
6,0.916988
7,0.927908
8,0.938424
9,0.947595
10,0.954277


Check the effect of different numbers if PCs on the outcome of the NN, testing n=2, 5 and 10. Use each n components to train and test the neural network, and compare the performance of each one

## Neural Network

In [ ]:
INPUT_SIZE = 69
OUTPUT_SIZE = len(output_columns)
ROWS = 1927

### n = 2

Train test split of data

In [ ]:
n=2
X = n_components[n][['pca'+ str(x) for x in range(n)]]
Y = combined[output_columns]

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=629) 

Model definition

In [ ]:
model = tf.keras.models.Sequential([
	tf.keras.layers.Dense(32, activation='relu', input_dim=len(X_train.columns)),
	tf.keras.layers.Dense(128, activation='relu'),
	tf.keras.layers.Dense(OUTPUT_SIZE)
])
model.compile(loss="mean_squared_error", optimizer="sgd", metrics="accuracy")
model.fit(X_train, y_train, epochs=500, batch_size=50);

Epoch 1/500
31/31 [==============================] - 0s 450us/step - loss: 28.9785 - accuracy: 0.4458
Epoch 2/500
31/31 [==============================] - 0s 320us/step - loss: 17.0623 - accuracy: 0.4685
Epoch 3/500
31/31 [==============================] - 0s 290us/step - loss: 14.0543 - accuracy: 0.4705
Epoch 4/500
31/31 [==============================] - 0s 312us/step - loss: 11.5523 - accuracy: 0.4705
Epoch 5/500
31/31 [==============================] - 0s 297us/step - loss: 9.8116 - accuracy: 0.4711
Epoch 6/500
31/31 [==============================] - 0s 291us/step - loss: 9.1869 - accuracy: 0.4666
Epoch 7/500
31/31 [==============================] - 0s 286us/step - loss: 8.8358 - accuracy: 0.4659
Epoch 8/500
31/31 [==============================] - 0s 330us/step - loss: 8.6591 - accuracy: 0.4679
Epoch 9/500
31/31 [==============================] - 0s 279us/step - loss: 8.6919 - accuracy: 0.4737
Epoch 10/500
31/31 [==============================] - 0s 317us/step - loss: 8.4276 - ac

In [ ]:
y_hat = model.predict(X_test)
print(y_hat[1])
y_test.head()

13/13 [==============================] - 0s 322us/step
[1.5092157e+00 1.2852345e+00 1.2939038e+01 1.1801120e+01 4.6359124e+00
 4.1302862e+00 5.1515560e+00 4.6354375e+00 1.0328215e+01 1.0850462e+01
 1.5869651e+00 1.8230746e+00 3.7820306e-02 3.2119211e-03]


,home_goals,away_goals,home_shots,away_shots,home_shots_on_target,away_shots_on_target,home_corners,away_corners,home_fouls,away_fouls,home_yellow_cards,away_yellow_cards,home_red_cards,away_red_cards
1489,1,5,13,13,5,7,7,3,4,6,0,1,0,0
643,0,0,21,8,4,1,11,7,11,9,4,3,0,0
1263,1,2,9,17,3,7,4,11,8,12,0,0,0,0
848,1,3,19,15,6,6,5,4,9,10,0,0,0,0
95,0,4,7,24,1,8,4,4,14,10,4,2,0,0


### n = 5

In [ ]:
n=5
X = n_components[n][['pca'+ str(x) for x in range(n)]]
Y = combined[output_columns]

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=629) 

In [ ]:
model = tf.keras.models.Sequential([
	tf.keras.layers.Dense(32, activation='relu', input_dim=len(X_train.columns)),
	tf.keras.layers.Dense(128, activation='relu'),
	tf.keras.layers.Dense(OUTPUT_SIZE)
])
model.compile(loss="mean_squared_error", optimizer="sgd", metrics="accuracy")
model.fit(X_train, y_train, epochs=500, batch_size=50);

Epoch 1/500
31/31 [==============================] - 0s 397us/step - loss: 25.0503 - accuracy: 0.3829
Epoch 2/500
31/31 [==============================] - 0s 333us/step - loss: 15.0505 - accuracy: 0.4653
Epoch 3/500
31/31 [==============================] - 0s 330us/step - loss: 12.9103 - accuracy: 0.4685
Epoch 4/500
31/31 [==============================] - 0s 320us/step - loss: 11.4134 - accuracy: 0.4698
Epoch 5/500
31/31 [==============================] - 0s 640us/step - loss: 10.2539 - accuracy: 0.4718
Epoch 6/500
31/31 [==============================] - 0s 320us/step - loss: 9.3698 - accuracy: 0.4750
Epoch 7/500
31/31 [==============================] - 0s 367us/step - loss: 8.9882 - accuracy: 0.4737
Epoch 8/500
31/31 [==============================] - 0s 323us/step - loss: 8.6497 - accuracy: 0.4770
Epoch 9/500
31/31 [==============================] - 0s 331us/step - loss: 8.7928 - accuracy: 0.4750
Epoch 10/500
31/31 [==============================] - 0s 343us/step - loss: 8.7332 - a

In [ ]:
y_hat = model.predict(X_test)
print(y_hat[1])
y_test.head()

### n = 10

In [ ]:
n=10
X = n_components[n][['pca'+ str(x) for x in range(n)]]
Y = combined[output_columns]

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=629) 

In [ ]:
model = tf.keras.models.Sequential([
	tf.keras.layers.Dense(32, activation='relu', input_dim=len(X_train.columns)),
	tf.keras.layers.Dense(128, activation='relu'),
	tf.keras.layers.Dense(OUTPUT_SIZE)
])
model.compile(loss="mean_squared_error", optimizer="sgd", metrics="accuracy")
model.fit(X_train, y_train, epochs=500, batch_size=50);

Epoch 1/500
31/31 [==============================] - 0s 2ms/step - loss: 26.2477 - accuracy: 0.4127
Epoch 2/500
31/31 [==============================] - 0s 1ms/step - loss: 14.6243 - accuracy: 0.4698
Epoch 3/500
31/31 [==============================] - 0s 581us/step - loss: 12.1046 - accuracy: 0.4757
Epoch 4/500
31/31 [==============================] - 0s 1ms/step - loss: 10.8859 - accuracy: 0.4724
Epoch 5/500
31/31 [==============================] - 0s 644us/step - loss: 10.0437 - accuracy: 0.4737
Epoch 6/500
31/31 [==============================] - 0s 837us/step - loss: 9.4945 - accuracy: 0.4757
Epoch 7/500
31/31 [==============================] - 0s 1ms/step - loss: 9.1451 - accuracy: 0.4757
Epoch 8/500
31/31 [==============================] - 0s 928us/step - loss: 8.9876 - accuracy: 0.4744
Epoch 9/500
31/31 [==============================] - 0s 1ms/step - loss: 8.7652 - accuracy: 0.4737
Epoch 10/500
31/31 [==============================] - 0s 2ms/step - loss: 8.6956 - accuracy: 0.4

In [ ]:
y_hat = model.predict(X_test)
print(y_hat[1])
y_test.head()

### n=15

In [ ]:
n=15
X = n_components[n][['pca'+ str(x) for x in range(n)]]
Y = combined[output_columns]

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=629) 

In [ ]:
model = tf.keras.models.Sequential([
	tf.keras.layers.Dense(32, activation='relu', input_dim=len(X_train.columns)),
	tf.keras.layers.Dense(128, activation='relu'),
	tf.keras.layers.Dense(OUTPUT_SIZE)
])
model.compile(loss="mean_squared_error", optimizer="sgd", metrics="accuracy")
model.fit(X_train, y_train, epochs=500, batch_size=50);

Epoch 1/500
31/31 [==============================] - 0s 410us/step - loss: 28.0814 - accuracy: 0.4062
Epoch 2/500
31/31 [==============================] - 0s 336us/step - loss: 14.0733 - accuracy: 0.4646
Epoch 3/500
31/31 [==============================] - 0s 309us/step - loss: 11.6436 - accuracy: 0.4659
Epoch 4/500
31/31 [==============================] - 0s 305us/step - loss: 10.4030 - accuracy: 0.4679
Epoch 5/500
31/31 [==============================] - 0s 279us/step - loss: 9.7650 - accuracy: 0.4672
Epoch 6/500
31/31 [==============================] - 0s 354us/step - loss: 9.4371 - accuracy: 0.4698
Epoch 7/500
31/31 [==============================] - 0s 392us/step - loss: 9.1480 - accuracy: 0.4698
Epoch 8/500
31/31 [==============================] - 0s 310us/step - loss: 8.9857 - accuracy: 0.4737
Epoch 9/500
31/31 [==============================] - 0s 312us/step - loss: 8.8351 - accuracy: 0.4718
Epoch 10/500
31/31 [==============================] - 0s 309us/step - loss: 8.7343 - ac

In [ ]:
y_hat = model.predict(X_test)
print(y_hat[1])
y_test.head()

### n=20

In [ ]:
n=20
X = n_components[n][['pca'+ str(x) for x in range(n)]]
Y = combined[output_columns]

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=629) 

In [ ]:
model = tf.keras.models.Sequential([
	tf.keras.layers.Dense(32, activation='relu', input_dim=len(X_train.columns)),
	tf.keras.layers.Dense(128, activation='relu'),
	tf.keras.layers.Dense(OUTPUT_SIZE)
])
model.compile(loss="mean_squared_error", optimizer="sgd", metrics="accuracy")
model.fit(X_train, y_train, epochs=500, batch_size=50);

Epoch 1/500
31/31 [==============================] - 0s 439us/step - loss: 27.6977 - accuracy: 0.3751
Epoch 2/500
31/31 [==============================] - 0s 342us/step - loss: 14.0264 - accuracy: 0.4653
Epoch 3/500
31/31 [==============================] - 0s 342us/step - loss: 11.6746 - accuracy: 0.4679
Epoch 4/500
31/31 [==============================] - 0s 365us/step - loss: 10.4352 - accuracy: 0.4692
Epoch 5/500
31/31 [==============================] - 0s 341us/step - loss: 9.7358 - accuracy: 0.4724
Epoch 6/500
31/31 [==============================] - 0s 342us/step - loss: 9.3105 - accuracy: 0.4724
Epoch 7/500
31/31 [==============================] - 0s 378us/step - loss: 9.0061 - accuracy: 0.4744
Epoch 8/500
31/31 [==============================] - 0s 306us/step - loss: 8.9488 - accuracy: 0.4770
Epoch 9/500
31/31 [==============================] - 0s 335us/step - loss: 8.8555 - accuracy: 0.4711
Epoch 10/500
31/31 [==============================] - 0s 351us/step - loss: 8.7985 - ac

In [ ]:
y_hat = model.predict(X_test)
print(y_hat[1])
y_test.head()

13/13 [==============================] - 1s 368us/step


The above experiments show us the accuracy of each model increases with an increased number of features up to a point, it seem somewhere between 15 and 20 features, where the accuracy on the training data peaks. I will use 15 principal components going forward for the investigations, as this seems to yield the highest accuracy of those tested here

In terms of predicting exact values for the match facts, I am not expecting exact matches between model and the actual results, just hoping for a close approximation, or at the very least, the correct side with the higher value, i.e. home team scores more goals than the away team is correct

# Next steps

- Tune parameters and layers on the 15 PC model to increase the accuracy
- Save the model
- Complete the notebook with all annotations and text